In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/4.link'):
    shutil.rmtree('./file/4.link')

os.makedirs('./file/4.link')

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                             'username': '186299u15',
                             'password': 'APMTeDoW14'}}

b = 0
while True:
    b += 1

    try:
        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                             data=json.dumps(data),
                             headers=get_header(),
                             #proxies=get_proxy(),
                             timeout=(10, 10))

        if resp.status_code == 200:
            json_ = resp.json()

            if json_['status'] == 200:
                key = json_['apiKey']
                break
    except KeyboardInterrupt:
        break
    except:
        continue

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/3.make'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/3.make/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)
    input_['Part Code'] = input_['Part Code'].astype(int)
    input_['Make No'] = input_['Make No'].astype(int)
    input_['Make Code'] = input_['Make Code'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error, key

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                data = {'getArticleLinkedAllLinkingTarget4': {'articleCountry': 'DE',
                                                              'articleId': int(input_.loc[a, 'Part Code']),
                                                              'country': 'DE',
                                                              'countryGroupFlag': False,
                                                              'lang': 'en',
                                                              'linkingTargetManuId': int(input_.loc[a, 'Make Code']),
                                                              'linkingTargetType': 'VOLB',
                                                              'provider': 40_000,
                                                              'withMainArticles': False}}
                
                b = 0
                while True:
                    b += 1
                    
                    try:
                        resp = requests.post('https://webservice.tecalliance.services/pegasus-3-0/services/TecdocToCatDLB.jsonEndpoint',
                                             data=json.dumps(data),
                                             headers=get_header(**{'x-api-key': key}),
                                             #proxies=get_proxy(),
                                             timeout=(10, 10))
                            
                        if resp.status_code == 200:
                            json_ = resp.json()
                
                            if json_['status'] == 401 and json_['statusText'].strip() == 'Access not allowed':
                                while True:
                                    try:
                                        data = {'getAPIKeyForUser': {'catalog': 'tecdocsw',
                                                                     'username': '186299u15',
                                                                     'password': 'APMTeDoW14'}}
                                        
                                        resp = requests.post('https://webservice.tecalliance.services/webcat30/v1/services/WebCat30WS.jsonEndpoint',
                                                             data=json.dumps(data),
                                                             headers=get_header(),
                                                             #proxies=get_proxy(),
                                                             timeout=(10, 10))
                                
                                        if resp.status_code == 200:
                                            json_ = resp.json()
                                
                                            if json_['status'] == 200:
                                                key = json_['apiKey']
                                                break
                                    except KeyboardInterrupt:
                                        break
                                    except:
                                        continue
                            elif json_['status'] == 200:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                list_link = json_['data']['array'][0]['articleLinkages']['array']

                # = = = = = = = = = = = = = = =

                if not list_link:
                    raise

                # = = = = = = = = = = = = = = =

                list_dict_link, index_start, index_end = [], 0, 25
                while True:
                    dict_link = {'array': []}
                    if index_end < len(list_link):
                        for link in list_link[index_start:index_end]:
                            dict_link['array'].append({'articleLinkId': link['articleLinkId'], 'linkingTargetId': link['linkingTargetId']})
                        list_dict_link.append(dict_link)
                        index_start, index_end = index_end, index_end + 25
                    else:
                        for link in list_link[index_start:]:
                            dict_link['array'].append({'articleLinkId': link['articleLinkId'], 'linkingTargetId': link['linkingTargetId']})
                        list_dict_link.append(dict_link)
                        break

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'No': input_.loc[a, 'No'],
                                        'Part Number': input_.loc[a, 'Part Number'],
                                        'Part Code': input_.loc[a, 'Part Code'],
                                        'Link Type': input_.loc[a, 'Link Type'],
                                        'Make No': input_.loc[a, 'Make No'],
                                        'Make': input_.loc[a, 'Make'],
                                        'Make Code': input_.loc[a, 'Make Code'],
                                        'Link No': [i+1 for i in range(len(list_dict_link))],
                                        'Json_Link': [json.dumps(dict_link) for dict_link in list_dict_link]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']} > {input_.loc[a, 'Make No']}.{input_.loc[a, 'Make']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(50):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No', 'Make No', 'Link No'],
                                                    ascending=[True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/4.link/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No', 'Make No'],
                                                ascending=[True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-link_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/4.link'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/4.link/{file}',
              f'''./file/4.link/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1134.18696 > 1.CITROËN
[剩余数量：1000] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1157.188643 > 1.MERCEDES-BENZ
[剩余数量：999] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1158.188647 > 3.VW
[剩余数量：998] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1172.19019 > 1.CITROËN
[剩余数量：997] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1171.19017 > 4.PEUGEOT
[剩余数量：996] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1172.19019 > 3.LANCIA
[剩余数量：995] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1142.187809 > 1.TOYOTA
[剩余数量：994] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1171.19017 > 2.FIAT
[剩余数量：993] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1164.188653 > 1.AUDI
[剩余数量：992] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1169.19013 > 1.CITROËN
[剩余数量：991] - [当前时间：16:05:16]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1172.19019 > 2.FIAT
[剩余数量：990] - [当前时间：16:05:16]

[状态：

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3695.42it/s]

Done ~
